In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os

# Define connected componnet removal

In [14]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [3]:
directory = Path('matt_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [5]:
len(files)

390

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [8]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[:]):
    mean = np.load(file)
    std = np.load(re.sub('mean','std',file))
    seg = np.zeros(mean.shape[1:])
    seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
    seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
    seg = seg.astype('int8')
    seg = (seg==1)*1
    seg = fill_holes(seg)
    np.save(re.sub('mean','seg',file),seg)
    #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
    #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [58:24<00:00,  8.99s/it]


# ANTs registration

## Upsample raw images

In [17]:
image_path = Path('../TBI')
images = list(image_path.glob('*_*/*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

486

In [ ]:
for i in tqdm(range(len(images))[::-1]):
    image = io.imread(images[i])
    image = np.swapaxes(image,0,1)
    image = np.swapaxes(image,1,3)
    image = resize(image,(2,507,507,252),preserve_range=True)
    zeros = np.zeros((1,507,507,252)).astype('uint16')
    image = np.append(image,zeros,axis = 0)
    image = image.astype('float16')
    io.imsave('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

 13%|██████████████████████▏                                                                                                                                                    | 63/486 [26:16<2:57:05, 25.12s/it]

## reshape numpy seg files and resave as tif

In [11]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [12]:
for i in tqdm(range(len(files_seg))):
    tmp = np.load(files_seg[i])
    tmp = np.reshape(tmp,(1,507,507,252))
    io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

  0%|                                                                                                                                                                                      | 0/390 [00:00<?, ?it/s]/tmp/ipykernel_1631580/3179607572.py:4: UserWarning:

matt_preds/XYZres100_0001_seg.tif is a low contrast image

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [01:45<00:00,  3.71it/s]


## Register images and transform masks

In [43]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_upsamp.tif'))
images = sorted([x.as_posix() for x in images])#[0:2]
images[8]

'matt_preds/XYZres032_upsamp.tif'

In [44]:
len(images)

395

In [ ]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('upsamp','seg',images[i])):
        fix = ants.image_read(re.sub('_0001','',images[i]))
        mov = ants.image_read(images[i])
        fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
        mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
        mytx = ants.registration(fixed = fix,
                                moving = mov,
                                type_of_transform = 'Rigid'
                                )
        warpedmask = ants.apply_transforms(fixed = fix_mask,
                                           moving = mov_mask,
                                           transformlist = mytx['fwdtransforms'],
                                           interpolator = 'nearestNeighbor'
                                          )
        io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
        io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
        #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

## Save Matlab .mat file of registered images

In [13]:
os.path.exists(re.sub('upsamp','seg',images[i]))

True

In [39]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])

In [40]:
for image in tqdm(images):
    if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':seg})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [00:20<00:00,  5.84it/s]


1.0

In [10]:
re.sub('_0001','',image)

'matt_preds/XYZres100_seg_warped.tif'